In [1]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
import re
from sklearn.preprocessing import RobustScaler
import scipy.stats as ss
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [2]:
os.listdir('C:/Users/froke/jupyter_notebook_files/anomaly_detection/EMEM算法/')

['.idea',
 '.ipynb_checkpoints',
 'baseModel.py',
 'ceshi',
 'dataloader',
 'datasets',
 'disagreement_KDDCup99.ipynb',
 'disagreement_parkinson.ipynb',
 'kddcup',
 'Parkinson',
 'Parkinson.tar.gz',
 'Parkinson_withoutdupl_75.txt']

In [3]:
with open("./Parkinson_withoutdupl_75.txt",'r') as f:
    data = f.readlines()
# print(data[28:-2])
col_names = []
list_ = []
for id_, line in enumerate(data):
    if line.startswith("@ATTRIBUTE"):
        colName = re.findall("@ATTRIBUTE '(.*?)'", line)[0]
        col_names.append(colName)
    if id_ >= 28 and id_<= len(data) - 1 and line != "\n":
        new_line = line.split(",")
        new_line = [x.strip() for x in new_line]
        new_line = [float(new_line[x]) for x in range(len(new_line)-1)] + [new_line[-1].replace("'","")]
        list_.append(new_line)
        
df = pd.DataFrame(list_, columns=col_names)
df['outlier'] = df.outlier.apply(lambda x: 1 if x =='yes' else 0)

In [4]:
df_sampled = df.sample(frac=1).reset_index(drop=True)
X = df_sampled.iloc[:,1:-1]
y_ground_truth = df_sampled.iloc[:,-1]

In [5]:
X

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,MDVP:APQ,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE
0,167.930,193.221,79.068,0.00442,0.000030,0.00220,0.00247,0.00661,0.04351,0.377,...,0.04246,0.06685,0.01280,22.468,0.619060,0.679834,-4.330956,0.262384,2.916777,0.285695
1,126.344,134.231,112.773,0.00448,0.000040,0.00131,0.00169,0.00393,0.02033,0.185,...,0.01614,0.03429,0.00474,25.030,0.507504,0.760361,-6.689151,0.291954,2.431854,0.105993
2,115.380,123.109,108.634,0.00332,0.000030,0.00160,0.00199,0.00480,0.01503,0.137,...,0.01133,0.02436,0.00401,26.005,0.405991,0.761255,-5.966779,0.197938,1.974857,0.184067
3,154.003,160.267,128.621,0.00183,0.000010,0.00076,0.00100,0.00229,0.01030,0.094,...,0.00871,0.01406,0.00243,28.409,0.263654,0.691483,-7.111576,0.144780,2.065477,0.093193
4,202.266,211.604,197.079,0.00180,0.000009,0.00093,0.00107,0.00278,0.00954,0.085,...,0.00719,0.01407,0.00072,32.684,0.368535,0.742133,-7.695734,0.178540,1.544609,0.056141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,157.447,163.267,149.605,0.00369,0.000020,0.00201,0.00197,0.00602,0.03272,0.283,...,0.02571,0.05439,0.01018,21.693,0.447285,0.705658,-6.247076,0.180528,2.344348,0.164916
191,102.273,142.830,85.902,0.00907,0.000090,0.00493,0.00461,0.01480,0.02814,0.272,...,0.02073,0.04736,0.03882,18.447,0.671378,0.674562,-2.929379,0.396746,2.560422,0.367233
192,96.106,108.664,84.510,0.00694,0.000070,0.00389,0.00415,0.01168,0.04024,0.364,...,0.02876,0.06799,0.01823,19.055,0.544805,0.770466,-4.441519,0.155097,2.645959,0.327978
193,153.046,175.829,68.623,0.00742,0.000050,0.00364,0.00432,0.01092,0.05517,0.542,...,0.05767,0.07413,0.03160,17.280,0.665318,0.719467,-3.949079,0.357870,3.109010,0.377429


- 孤立森林不用归一化
- LOF 要做归一化

In [6]:
ro_scaler = RobustScaler()
X_scaled_robust = ro_scaler.fit_transform(X)

# 打分矩阵

let's say we have 200 detectors

In [7]:
score_matrix = np.zeros([X.shape[0], 10])

# 孤立森林

In [8]:
isolate_clfs = []
for i in range(5):
    n_est = 100
    clf = IsolationForest(n_estimators=n_est,contamination=0.5)
    clf.fit(X)
    output_score = abs(clf.score_samples(X))
    score_matrix[:, i] = output_score
    isolate_clfs.append(clf)

# LOF with scaled data

In [9]:
lof_clfs = []
for i in range(5):
    n_nb = np.random.choice(range(5,20))
    clf = LocalOutlierFactor(n_neighbors=n_nb)
    clf.fit(X_scaled_robust)
    score_ = abs(clf.negative_outlier_factor_)
    score_matrix[:, i+5] = score_
    lof_clfs.append(clf)

In [10]:
score_matrix

array([[0.43799181, 0.42613304, 0.43163887, ..., 1.0278738 , 1.05123249,
        1.04581356],
       [0.40336464, 0.39718348, 0.41040454, ..., 1.01311337, 1.01138897,
        1.00363289],
       [0.38800924, 0.38786862, 0.38477854, ..., 0.9519595 , 0.9778155 ,
        0.97162976],
       ...,
       [0.43030762, 0.4183279 , 0.44346492, ..., 0.99949928, 0.98164314,
        0.98238443],
       [0.4682963 , 0.48196982, 0.48466748, ..., 1.12453622, 1.10219217,
        1.1221077 ],
       [0.41914333, 0.43119986, 0.43030214, ..., 1.27532227, 1.34168111,
        1.3689561 ]])

In [11]:
score_matrix.shape

(195, 10)

# disagreement matrix

## 1. ranking disagreement

In [12]:
rank_matrix = np.zeros([len(X), 10])

In [13]:
for i in range(10):
    # rank by each column and get its rank_position
    rank_matrix[:,i] = ss.rankdata(score_matrix[:,i])
rank_matrix[:5]

array([[127., 103., 103., 124., 114.,  87.,  87.,  76.,  93.,  87.],
       [ 48.,  32.,  61.,  48.,  50.,  33.,  33.,  63.,  49.,  39.],
       [  8.,   6.,   5.,   2.,   7.,  25.,  25.,   2.,  11.,   4.],
       [161., 163., 150., 150., 153.,  96.,  96., 122., 101., 102.],
       [178., 175., 166., 172., 179., 125., 125., 146., 137., 137.]])

calculate standard deviation

In [14]:
std_record = np.zeros(len(X))
for i in range(195):
    std_record[i] = np.std(rank_matrix[i,:])

In [15]:
std_record

array([16.28158469, 10.62261738,  8.16394512, 27.15952871, 21.05706532,
       29.59408725, 44.93951491, 17.93014222, 19.01157542,  9.63587048,
        9.4       , 47.16820963, 25.83195695, 22.70880886, 20.50097559,
       48.31821603, 11.37585162, 10.05236291, 13.00038461, 16.31563667,
       44.33914749, 19.21587885, 14.07977273,  5.02394267, 10.16070864,
       39.5494627 ,  7.30821456, 11.85748709, 19.56425312,  7.90189851,
       19.54098258, 11.98540779, 11.99208072,  3.84837628, 21.54738963,
       14.81080687, 31.39442626, 52.47894816,  5.78013841,  0.78102497,
        0.48989795, 32.08426406, 38.90514105, 19.47844963, 28.61118662,
       14.48067678, 16.09347694, 25.30533541, 18.46862204, 49.03886214,
       12.88759093, 13.43465668, 20.39338128,  8.21035931, 17.11256848,
       28.40932945, 16.79553512, 37.61927697,  5.62138773, 37.06426851,
       32.64368239, 29.98933144, 26.09137789, 45.43522862,  5.90254183,
        9.0465463 , 27.48817928, 11.83596215, 35.15807162, 17.25

get top 5 and get the middle score

In [16]:
std_max_rows = np.argsort(std_record)[-5:]
std_max_rows

array([174, 181, 145,  77, 184], dtype=int64)

In [17]:
print(rank_matrix[std_max_rows])
print()
print(std_record[std_max_rows])

[[ 75.  27.  53.  51.  44. 156. 156. 163. 151. 144.]
 [ 52.   7.  13.  14.  35. 152. 152. 125. 104.  78.]
 [150. 158. 160. 159. 156.  59.  59.  10.  53.  73.]
 [188. 187. 187. 184. 186.  48.  48. 186. 185. 187.]
 [ 29.  63.  20.  37.  25. 176. 176. 137. 116. 100.]]

[53.32729132 54.30801046 55.08547903 55.31039685 58.40967386]


so, we get the score threshold:

In [18]:
score_matrix[std_max_rows[-1]]

array([0.39758803, 0.40721499, 0.3945805 , 0.40154023, 0.39376606,
       1.22277357, 1.22277357, 1.13470223, 1.07944098, 1.05715626])

# comparison
## 1. SD
## 2. IQR
## 3. MAD
## 4. 2T
## 5. clever standard deviation
## 6. disagreement

----

lof_clfs, isolate_clfs

In [19]:
perform_records_base = {"SD":{}, "IQR":{}, "MAD":{}, "2T":{}, "Disagreement":{}}
for i in ["eucledian", "scaled_eucledian",]+ ["clf_"+str(j+1) for j in range(10)]:
    for key in perform_records_base.keys():
        perform_records_base[key][i]=0
acc_perform = perform_records_base.copy()
f1_perform = perform_records_base.copy()

### 1. SD_model

In [20]:
for i in range(score_matrix.shape[1]):
    high_limit = np.mean(score_matrix[:, i]) + 3*np.std(score_matrix[:, i])
    y_predict = score_matrix[:,i]>high_limit
    y_predict = [1 if j else 0 for j in y_predict]
    f1 = f1_score(y_ground_truth, y_predict)
    acc = accuracy_score(y_ground_truth, y_predict)
    f1_perform["SD"]["clf_"+str(i+1)] = f1
    acc_perform["SD"]["clf_"+str(i+1)] = acc

### IQR model


In [21]:
for i in range(score_matrix.shape[1]):
    iqr = np.percentile(score_matrix[:,i],75) - np.percentile(score_matrix[:,i],25)
    y_predict = score_matrix[:,i]> np.percentile(score_matrix[:,i],75)+1.5*iqr
    y_predict = [1 if j else 0 for j in y_predict]
    f1 = f1_score(y_ground_truth, y_predict)
    acc = accuracy_score(y_ground_truth, y_predict)
    f1_perform["IQR"]["clf_"+str(i+1)] = f1
    acc_perform["IQR"]["clf_"+str(i+1)] = acc
    

### MAD model

In [22]:
for i in range(score_matrix.shape[1]):
    median_ = np.median(score_matrix[:,i])
    mad = 1.4826*np.median(np.abs(score_matrix[:,i]-median_))
    y_predict = score_matrix[:,i]> 3*mad
    y_predict = [1 if j else 0 for j in y_predict]
    f1 = f1_score(y_ground_truth, y_predict)
    acc = accuracy_score(y_ground_truth, y_predict)
    f1_perform["MAD"]["clf_"+str(i+1)] = f1
    acc_perform["MAD"]["clf_"+str(i+1)] = acc

### disagreement

In [27]:
for i in range(score_matrix.shape[1]):
    y_predict = score_matrix[:,i] > score_matrix[std_max_rows[-1]][i]
    y_predict = [1 if j else 0 for j in y_predict]
    f1 = f1_score(y_ground_truth, y_predict)
    acc = accuracy_score(y_ground_truth, y_predict)
    f1_perform["Disagreement"]["clf_"+str(i+1)] = f1
    acc_perform["Disagreement"]["clf_"+str(i+1)] = acc

In [28]:
f1_perform

{'SD': {'eucledian': 0,
  'scaled_eucledian': 0,
  'clf_1': 0.2717948717948718,
  'clf_2': 0.26666666666666666,
  'clf_3': 0.26666666666666666,
  'clf_4': 0.26666666666666666,
  'clf_5': 0.26666666666666666,
  'clf_6': 0.2564102564102564,
  'clf_7': 0.2564102564102564,
  'clf_8': 0.2564102564102564,
  'clf_9': 0.26153846153846155,
  'clf_10': 0.26666666666666666},
 'IQR': {'eucledian': 0,
  'scaled_eucledian': 0,
  'clf_1': 0.3076923076923077,
  'clf_2': 0.31794871794871793,
  'clf_3': 0.30256410256410254,
  'clf_4': 0.29743589743589743,
  'clf_5': 0.2923076923076923,
  'clf_6': 0.28205128205128205,
  'clf_7': 0.28205128205128205,
  'clf_8': 0.28205128205128205,
  'clf_9': 0.30256410256410254,
  'clf_10': 0.3230769230769231},
 'MAD': {'eucledian': 0,
  'scaled_eucledian': 0,
  'clf_1': 0.7538461538461538,
  'clf_2': 0.7538461538461538,
  'clf_3': 0.7538461538461538,
  'clf_4': 0.7538461538461538,
  'clf_5': 0.7538461538461538,
  'clf_6': 0.7538461538461538,
  'clf_7': 0.753846153846153

## 2T-SD

In [25]:
for i in range(score_matrix.shape[1]):
    high_limit = np.mean(score_matrix[:, i]) + 3*np.std(score_matrix[:, i])
    std2 = np.std(score_matrix[:,i]<=high_limit)
    high_limit2 = np.mean(score_matrix[score_matrix[:,i]<=high_limit, i]) + 3*std2
    y_predict = score_matrix[:,i]> high_limit2
    y_predict = [1 if j else 0 for j in y_predict]
    f1 = f1_score(y_ground_truth, y_predict)
    acc = accuracy_score(y_ground_truth, y_predict)
    f1_perform["2T"]["clf_"+str(i+1)] = f1
    acc_perform["2T"]["clf_"+str(i+1)] = acc

In [26]:
f1_perform

{'SD': {'eucledian': 0,
  'scaled_eucledian': 0,
  'clf_1': 0.2717948717948718,
  'clf_2': 0.26666666666666666,
  'clf_3': 0.26666666666666666,
  'clf_4': 0.26666666666666666,
  'clf_5': 0.26666666666666666,
  'clf_6': 0.2564102564102564,
  'clf_7': 0.2564102564102564,
  'clf_8': 0.2564102564102564,
  'clf_9': 0.26153846153846155,
  'clf_10': 0.26666666666666666},
 'IQR': {'eucledian': 0,
  'scaled_eucledian': 0,
  'clf_1': 0.3076923076923077,
  'clf_2': 0.31794871794871793,
  'clf_3': 0.30256410256410254,
  'clf_4': 0.29743589743589743,
  'clf_5': 0.2923076923076923,
  'clf_6': 0.28205128205128205,
  'clf_7': 0.28205128205128205,
  'clf_8': 0.28205128205128205,
  'clf_9': 0.30256410256410254,
  'clf_10': 0.3230769230769231},
 'MAD': {'eucledian': 0,
  'scaled_eucledian': 0,
  'clf_1': 0.7538461538461538,
  'clf_2': 0.7538461538461538,
  'clf_3': 0.7538461538461538,
  'clf_4': 0.7538461538461538,
  'clf_5': 0.7538461538461538,
  'clf_6': 0.7538461538461538,
  'clf_7': 0.753846153846153